In [1]:
import librosa as lr
import numpy as np
import librosa.display
import pyaudio
import wave
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import models
model = keras.models.load_model("model_VGG19_cough_acc88.75_best_50epoch.h5")
new_datagen = ImageDataGenerator(rescale=1./255)
from matplotlib.backends.backend_qt4agg import FigureCanvasQTAgg as FigureCanvas
from datetime import datetime
import pyrebase
from datetime import timedelta

<ipython-input-1-40f92a532c50>:14: MatplotlibDeprecationWarning: 
The matplotlib.backends.backend_qt4agg backend was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
  from matplotlib.backends.backend_qt4agg import FigureCanvasQTAgg as FigureCanvas


In [2]:
import speech_recognition as aud
r=aud.Recognizer()
#from Crypto.PublicKey import RSA

In [3]:
config = {
  "apiKey": "AIzaSyAWBjr4oXVzbvxvxdBoRKkRRgA",
  "authDomain": "Cough_Detection.firebaseapp.com",
  "databaseURL": "https://Cough_Detection.firebaseio.com/",
  "storageBucket": "gs://Cough_Detection.appspot.com",

}
firebase = pyrebase.initialize_app(config)
db = firebase.database()


In [4]:
#Final with consecutive coughing
i=0


def record():
    global i
    while True:
        CHUNK = 1024
        FORMAT = pyaudio.paInt16
        CHANNELS = 1
        RATE = 44100
        RECORD_SECONDS = 2
        WAVE_OUTPUT_FILENAME = "output.wav"

        p = pyaudio.PyAudio()
        import time
        start = time.time()

        stream = p.open(format=FORMAT,
                        channels=CHANNELS,
                        rate=RATE,
                        input=True,
                        frames_per_buffer=CHUNK)

        #print("* recording")

        frames = []

        for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
            data = stream.read(CHUNK)
            frames.append(data)

        #print("* done recording")
        stream.stop_stream()
        stream.close()
        p.terminate()

        wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(p.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))
        i=1

#starting preprocessing
new_val = 0
prev_val = 0
consicutive = 1
concount = 0
now = datetime.now()
end_timer = now.now()
def process():
    global new_val
    global prev_val
    global i
    tlimit = timedelta(0,10)
    global consicutive
    global concount
    global end_timer
    while True:
        if i==1:
            y, sr = librosa.load('output.wav',duration=3)
            ps = librosa.feature.melspectrogram(y=y, sr=sr)
            ps_db= librosa.power_to_db(ps, ref=np.max)

            fig = plt.Figure(figsize= (3,3))
            canvas = FigureCanvas(fig)
            ax = fig.add_subplot(111)
            ax.set_axis_off()
            abc = librosa.display.specshow(ps_db,ax=ax, x_axis='time', y_axis='mel')
            fig.savefig(r'C:\Users\**\Pictures\Saved Pictures\cough\cough15.png')
            new_dir=r'C:\Users\**\Pictures\Saved Pictures'

            #prediction started
            new_generator = new_datagen.flow_from_directory(
                new_dir,
                target_size=(150, 150),
                batch_size=1,
                class_mode='binary'
                )
            y_pred = model.predict_classes(new_generator)
            newval=y_pred
            #print(newval,prev_val)
            if prev_val!=newval:

                if y_pred==0:
                    print('Didnt Cough')
                    data=db.child("gi").set(0)
                    
                    #google api
#                     try:
#                         audio=aud.AudioFile('output.wav')
#                         with audio as source:
#                             audio=r.record(source)
#                         val=r.recognize_google(audio)
#                         print(val,end='\n')
#                     except:
#                         pass

#                     google api ends
                if y_pred==1:
                    
                    now = datetime.now()
                    #newline starts
                    
                    if concount == 0:
                        start_timer = now.now()
                        concount = 1
                    else:
                        consicutive += 1
                        end_timer = now.now()
                    diff = end_timer-start_timer
                    if diff<tlimit:
                        start_timer = now.now()
                        current_time1 = now.strftime("%H:%M:%S")
                        data=db.child("consecutive cough").child("time").set(current_time1)
                        data=db.child("consecutive cough").child("cough").set(consicutive)
                        
                        print(f"coughed {consicutive} time in a row")
                    else:
                        concount = 0
                        consicutive = 1
                        
                    
                    current_time = now.strftime("%H:%M:%S")
                    print('Cough Detected')
                    print("last Cough time", current_time)
                    data=db.child("gi").set(1)
                    data=db.child("last_cough").set(current_time)

                #stop = time.time()
                #print("The time of the run:", stop - start)
                i=0
            else:
                #print('korlam pass')
                pass
            if y_pred==0:
                try:
                    audio=aud.AudioFile('output.wav')
                    with audio as source:
                        audio=r.record(source)
                    val=r.recognize_google(audio)
                    print(val)
                    if "help" in val:
                        now = datetime.now()
                        current_time = now.strftime("%H:%M:%S")
                        data=db.child("last_help").set(current_time)
                except:
                    pass

                
            prev_val = newval


In [ ]:
import threading
import time
thread1 = threading.Thread(target=record)
thread1.start()

thread2 = threading.Thread(target=process)
thread2.start()